# Detecting terrestrial heatwaves in Australia  

The purpose of this notebook is to take in daily tmax data, along with previously computed mean climatology and threshold values, process these and output all the THW events that occur in the dataset

First, we import the data and required modules

# Importing packages and defining functions

In [1]:
from dask.distributed import LocalCluster, Client
import dask.array
import datetime
from datetime import date 
from datetime import datetime
import glob
import numpy as np
import scipy.ndimage as ndimage
from scipy.ndimage.measurements import label, find_objects
import xarray as xr

In [2]:
import os
import dask.distributed
threads_per_worker = 1
try:
    c # Already running
except NameError:
    c = dask.distributed.Client(
        n_workers=4,
        threads_per_worker=threads_per_worker,
        memory_limit=f'{3.9*threads_per_worker}gb'
    )
c

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.10/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46137 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:39767 Dashboard: http://127.0.0.1:46137/status,Cluster Workers: 4 Cores: 4 Memory: 15.60 GB


This function will make the climatology and threshold repeatable so that the shapes of climatology, threshold and observation are equal.

In [3]:
def copy_clim(target_da,input_da):
    t_time=target_da["time"].dt.dayofyear[0]
    target_da[:] = input_da.sel(dayofyear=t_time)
    return(target_da)

The function 'calc_severity()' calculates severity based on the severity index introduced by Hobday et al (2018) for categorizing marine heatwaves.

This function merges together the chunks along the time dimension by setting the 'time' chunk size to None, so that the resulting chunks each contain the full time series for that location. To run the severity calculaion on each timeseries chunk, '.map_blocks()' is used so that the calculation can be done in parallel.

In [4]:
def calc_severity(data, new_climatology, new_threshold):

    threshold_anomaly = new_threshold - new_climatology
    
    # Ignore divide by zero errors
    nperr = np.seterr(divide='ignore')

    def calc_severity_helper(da, *args, **kwargs):
        if da.size == 0:
            return da
    
        coords = {}
        for k,v in obs_aus_tmax.coords.items():
            if k == 'time':
                continue
            coords[k] = slice(v[0], v[-1])
    
        anomaly = da - new_climatology.sel(coords)
        severity = anomaly / threshold_anomaly.sel(coords)
        return severity

    data = data.chunk({'time': None})
    r = data.map_blocks(calc_severity_helper)
    
    np.seterr(**nperr)
    
    return r

This function returns values with at least n contiguous points around them. I am using n=3, to define a terrestrial heatwave as an event where the daily tmax exceeds the 90th percentile for at least 3 consecutive days (Perkins and Alexander, 2013)

In [5]:
def atleastn(da, n, dim='time'):

    def atleastn_helper(array, n, axis):
        count = np.zeros_like(np.take(array, 0,axis=axis), dtype='i4')
        mask = np.empty_like(np.take(array, 0,axis=axis), dtype='bool')
        mask = True
    
        for i in range(array.shape[axis]):
            array_slice = np.take(array, i, axis=axis)
        
            # Increase the count when there is a valid value, reset when there is not
            # This was initially set to 0, now I have changed it to 1 to detect only valid heatwave days 
            # The previous way was fine as long as I masked values less than or equal to 1, and they were white on the colour bar
            count = np.where(array_slice > 1, count + 1, 0)
        
            # Add new points when the contiguous count exceeds the threshold
            mask = np.where(count >= n, False, mask)
            
        out_slice = np.take(array, array.shape[axis]//2, axis=axis)
        return np.where(mask, np.nan, out_slice)
    
    def atleastn_dask_helper(array, axis, **kwargs):
        r = dask.array.map_blocks(atleastn_helper, array, drop_axis=axis, axis=axis, n=n, dtype=array.dtype)
        return r
    
    if isinstance(da.data, dask.array.Array):
        reducer = atleastn_dask_helper
    else:
        reducer = atleastn_helper
        
    return da.rolling({dim: n*2-1}, center=True, min_periods=n).reduce(reducer, n=n)


## Opening files and loading data

In [6]:
mean_climatology = xr.open_dataarray('/g/data/e14/cp3790/Charuni/NCI/ERA5-T2M/climatology-australia.nc')
threshold = xr.open_dataarray('/g/data/e14/cp3790/Charuni/NCI/ERA5-T2M/threshold-australia.nc')

In [7]:
files = sorted(glob.glob('/g/data/e14/cp3790/Charuni/NCI/ERA5-T2M/era5_dailytmax_*.nc'))

obs_aus = (xr.open_mfdataset(files, combine='nested', concat_dim='time', chunks={'latitude': 10})
           .sel(time=slice('1982', '2018'), longitude=slice(113, 154), latitude=slice(-10, -44)))
obs_aus_tmax = obs_aus["dailytmax"] - 273.15
obs_aus_tmax.attrs['units'] = 'deg C'

Making the mean climatology and 90th percentile climatology (threshold) repeatable so that their sizes are compatible with the size of the obs_aus_tmax data using the 'copy_clim' function

In [ ]:
new_climatology = xr.DataArray(np.empty(obs_aus_tmax.shape),
                               coords=obs_aus_tmax.coords,
                               dims=obs_aus_tmax.dims,
                               name="climatology")
new_climatology = new_climatology.groupby(new_climatology.time.dt.dayofyear).apply(copy_clim,input_da=mean_climatology)

In [9]:
new_threshold= xr.DataArray(np.empty(obs_aus_tmax.shape),
                               coords=obs_aus_tmax.coords,
                               dims=obs_aus_tmax.dims,
                               name="threshold")
new_threshold=new_threshold.groupby(new_threshold.time.dt.dayofyear).apply(copy_clim,input_da=threshold)

## Calculations 

Calculating severity using 'map_blocks()', which enables the severity calculation to run on each timeseries chunk, facilitating parallel computation 

In [10]:
thw_severity = calc_severity(obs_aus_tmax, new_climatology, new_threshold)

We are only interested in the days where the daily tmax has exceeded the threshold. If severity > 1, this means that daily tmax > threshold (from the definition of severity). 

In [11]:
%%time
%matplotlib inline

candidates = thw_severity.where(thw_severity > 1)

CPU times: user 300 ms, sys: 74 ms, total: 374 ms
Wall time: 1.1 s


## Saving to netCDF

All the terrestrial heatwave days across Australia from 1982-2018 will be stored in filtered_severity_final_new_mask.nc 

In [12]:
%%time

# Next, we mask out points where there are less than 3 contiguous points in the time dimension
oscar = atleastn(candidates, n=3)

CPU times: user 41.5 ms, sys: 1.62 ms, total: 43.1 ms
Wall time: 42.3 ms


In [ ]:
xr.Dataset({'severity': oscar}).to_netcdf('/g/data/e14/cp3790/Charuni/filtered_severity_final_new_mask_CP.nc',
                                              encoding={'severity': 
                                                        {'chunksizes': (100, oscar.shape[1], oscar.shape[2]),
                                                         'zlib': True,
                                                         'shuffle': True, 
                                                         'complevel': 2}})   

# compression level (complevel) up to 6 is fine, >6 and it starts giving trouble 